# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Frameworks</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


## Redes Neurais Convolucionais

A Rede Neural Convolucional (CNN) é uma tecnologia de rede neural que impactou profundamente a área de Visão Computacional. Fukushima (1980) introduziu o conceito original de uma rede neural convolutiva, e LeCun, Bottou, Bengio & Haffner (1998) melhoraram muito este trabalho. A partir desta pesquisa, Yan LeCun apresentou a famosa arquitetura de rede neural LeNet-5. 

Existem similaridades e diferenças entre as CNNs e outros modelos de DNNs:

Normalmente usamos a classificação, embora a regressão ainda seja uma opção.
A entrada para a rede neural agora é 3D (altura, largura, cor)
Os dados não são transformados; sem zscores ou variáveis dummy.
O tempo de processamento é muito maior.
Agora temos diferentes camadas: camadas densas (como antes), camadas de convolução e camadas de Max Pooling.
Os dados não chegarão mais como arquivos CSV. O TensorFlow fornece alguns utilitários para ir diretamente da imagem para o input de uma rede neural.

Embora a Visão Computacional use principalmente CNNs, essa tecnologia possui aplicações em outras áreas. É possível utilizar CNNs em dados não visuais, mas nesse caso você deve encontrar uma maneira de codificar seus dados para que possa imitar as propriedades dos dados visuais. As CNNs são semelhantes à arquitetura de Mapas Auto Organizáveis (SOM - Self Organizing Maps). A ordem dos elementos vetoriais é crucial para o treinamento. Em contraste, a maioria das redes neurais que não são CNNs ou SOMs tratam seus dados de entrada como um vetor de valores, e a ordem em que você organiza as características de entrada neste vetor é irrelevante. Para esses tipos de redes neurais, você não pode alterar a ordem depois de ter treinado a rede. Em outras palavras, CNNs e SOMs não seguem o tratamento padrão de vetores de entrada.

Normalmente, uma rede neural feedforward comum cria todas as possíveis conexões de peso entre duas camadas. Na terminologia de aprendizagem profunda, nos referimos a essas camadas como camadas densas. Além de não representar todo o peso possível, as redes neurais convolutivas também compartilharão pesos.

## Datasets Para Visão Computacional

## CIFAR Data Set

Os datasets [CIFAR-10 and CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html) também são frequentemente usados pela comunidade de pesquisa de rede neural.

![CIFAR](imagens/cifar.png "CIFAR")

O conjunto de dados CIFAR-10 contém imagens divididas em 10 classes. O conjunto de dados CIFAR-100 contém 100 classes em uma hierarquia.

## Feature Learning e Classificação

![CNN](imagens/cnn.jpg "CNN")

In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# Para sumarizar o modelo
!pip install -q torchsummary

In [4]:
# Imports
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

torchvision 0.5.0
numpy       1.18.2
torch       1.4.0
Data Science Academy


## Arquitetura do Modelo

https://pytorch.org/docs/stable/nn.html#conv2d

![CNN](imagens/convolution.gif "Convolução")

![CNN](imagens/stride.png "Stride")

![CNN](imagens/padding.png "Padding")

In [6]:
# Exemplo de Camada de Convolução
nn.Conv2d(3, 16, 3, stride = 2, padding = (1,2))

Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 2))

In [7]:
# Classe Para o Modelo
class ModeloCNN(nn.Module):
    
    # Construtor da Classe
    def __init__(self):
        super().__init__()
        
        # Camada de Entrada
        self.conv1 = nn.Conv2d(3, 16, 3, padding = 1)
        
        # Primeira Camada Oculta
        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        
        # Segunda Camada Oculta
        self.conv3 = nn.Conv2d(32, 64, 3, padding = 1)
        
        # Camada de Pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Transformação Linear 1 (Camada Densa)
        self.linear1 = nn.Linear(64 * 4 * 4, 512)
        
        # Transformação Linear 2 (Camada Densa)
        self.linear2 = nn.Linear(512, 10) 
        
        # Camada de Dropout Para Regularização
        self.dropout = nn.Dropout(p = 0.3)
    
    # Método de Avanço da Rede
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        return x

## Preparando as Transformações Para os Dados

In [8]:
# Transformações Para os Dados de Entrada
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

## Carregando e Dividindo os Dados em Treino, Teste e Validação

In [9]:
# Carrega os Dados de Treino
train_data = datasets.CIFAR10('CIFAR10', train = True, download = True, transform = transformations)

Files already downloaded and verified


In [10]:
# Carrega os Dados de Teste
test_data = datasets.CIFAR10('CIFAR10', train = False, download = True, transform = transformations)

Files already downloaded and verified


In [11]:
# Tamanho dos Datasets
len(train_data), len(test_data)

(50000, 10000)

In [12]:
# Tamanho Para Dados de Validação
validation_size = 0.2

In [13]:
# Tamanho Para Dados de Treino
training_size = len(train_data)

In [14]:
# Índice 
indices = list(range(training_size))
np.random.shuffle(indices)
index_split = int(np.floor(training_size * validation_size))

In [15]:
# Índices Para Dados de Treino e Validação
validation_indices, training_indices = indices[:index_split], indices[index_split:]

In [16]:
# Amostras de Treino e Validação
training_sample = SubsetRandomSampler(training_indices)
validation_sample = SubsetRandomSampler(validation_indices)

In [17]:
# Hiperparâmetros
batch_size = 16
n_epochs = 30
learning_rate = 0.01

In [18]:
# Gera as Amostras Finais de Dados
train_loader = DataLoader(train_data, batch_size = batch_size, sampler = training_sample)
valid_loader = DataLoader(train_data, batch_size = batch_size, sampler = validation_sample)
test_loader = DataLoader(train_data, batch_size = batch_size)

## Otimização do Modelo

In [19]:
# Cria o Modelo (Instância da Classe)
model = ModeloCNN()

In [20]:
model

ModeloCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [21]:
# Identifica o Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device.type

'cuda'

In [22]:
# Envia o Modelo para a GPU
model = model.to(device)

In [23]:
# Define a Função de Custo Baseada em Cross Entropia
criterion = nn.CrossEntropyLoss()

In [24]:
# Cria o Otimizador (Backward)
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [25]:
# Função Para Acurácia
def accuracy(preds, y):
    _, pred = torch.argmax(preds, 1)
    correct = pred.eq(target.data.view_as(pred))
    acc = correct.sum()/len(correct)
    return acc

## Treinamento

In [26]:
# Looop
for epoch in range(1, n_epochs+1):
    
    # Inicializa o Erro a Cada Epoch
    train_loss = 0.0
    valid_loss = 0.0
    
    # Cria Objeto de Treinamento do Modelo
    model.train()
    
    # Para Cada Batch de Dados Executa o Treinamento
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    # Cria Objeto de Avaliação do Modelo    
    model.eval()
    
    # Para Cada Batch de Dados Executa a Avaliiação
    for batch_idx, (data, target) in enumerate(valid_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
        
    # Calcula o Erro na Epoch
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    # Print
    print(f'| Epoch: {epoch:02} | Erro em Treino: {train_loss:.3f} | Erro em Validação: {valid_loss:.3f} |')

| Epoch: 01 | Erro em Treino: 2.089 | Erro em Validação: 1.765 |
| Epoch: 02 | Erro em Treino: 1.672 | Erro em Validação: 1.540 |
| Epoch: 03 | Erro em Treino: 1.512 | Erro em Validação: 1.394 |
| Epoch: 04 | Erro em Treino: 1.415 | Erro em Validação: 1.313 |
| Epoch: 05 | Erro em Treino: 1.347 | Erro em Validação: 1.256 |
| Epoch: 06 | Erro em Treino: 1.283 | Erro em Validação: 1.175 |
| Epoch: 07 | Erro em Treino: 1.226 | Erro em Validação: 1.144 |
| Epoch: 08 | Erro em Treino: 1.182 | Erro em Validação: 1.114 |
| Epoch: 09 | Erro em Treino: 1.143 | Erro em Validação: 1.049 |
| Epoch: 10 | Erro em Treino: 1.110 | Erro em Validação: 1.033 |
| Epoch: 11 | Erro em Treino: 1.082 | Erro em Validação: 1.008 |
| Epoch: 12 | Erro em Treino: 1.054 | Erro em Validação: 0.988 |
| Epoch: 13 | Erro em Treino: 1.027 | Erro em Validação: 0.955 |
| Epoch: 14 | Erro em Treino: 1.004 | Erro em Validação: 0.936 |
| Epoch: 15 | Erro em Treino: 0.981 | Erro em Validação: 0.918 |
| Epoch: 16 | Erro em Tre

In [27]:
# Salva o modelo treinado
torch.save(model.state_dict(), "modelos/cifar10.pth")

# Fim